In [19]:
!pip install --upgrade psycopg2-binary

  Using cached psycopg2_binary-2.9.9-cp310-cp310-win_amd64.whl (1.2 MB)
  Attempting uninstall: psycopg2-binary
    Found existing installation: psycopg2-binary 2.8.6
    Uninstalling psycopg2-binary-2.8.6:
      Successfully uninstalled psycopg2-binary-2.8.6


In [1]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, EndpointConnectionError
import psycopg2
from psycopg2 import sql
import csv
from dotenv import load_dotenv
import os



In [2]:
# Creating an S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Bucket name and the path to upload the file
bucket_name = "d2b-internal-assessment-bucket"
key = "analytics_export/joshodey2178/"

In [3]:
# Load environmental variables from the .env file
load_dotenv("C:\\Users\\USER\\Documents\\Data2Bots Assessment\\.env")

# Get the environmental variables
PG_USERNAME = os.getenv("PG_USERNAME")
PG_PASS = os.getenv("PG_PASS")
PG_HOST = os.getenv("PG_HOST")
PG_DB = os.getenv("PG_DB")

In [4]:
# PostgreSQL database connection parameters
db_params = {
    'host': PG_HOST,
    'port': 5432,
    'database': PG_DB,
    'user': PG_USERNAME,
    'password': PG_PASS
}


connection = psycopg2.connect(**db_params)
cursor = connection.cursor()

In [11]:
Expo

# Query to get the list of tables in the schema
tables_query = f"SELECT table_name FROM information_schema.tables WHERE table_schema = 'joshodey2178_analytics';"
cursor.execute(tables_query)
tables = cursor.fetchall()

# Loop through each table and get ready in csv
for table in tables:
    table_name = table[0]
    csv_file_path = f"C:\\Users\\USER\\Documents\\Data2Bots Assessment\\table_exports\\{table_name}.csv"
    
    # Fetch data from the PostgreSQL table
    data_query = f"SELECT * FROM joshodey2178_analytics.{table_name};"
    cursor.execute(data_query)
    result = cursor.fetchall()
    
    # Write data to a CSV file
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Write the header (column names)
        csv_writer.writerow([desc[0] for desc in cursor.description])
        # Write the data rows
        csv_writer.writerows(result)

    print(f"Data from table {table_name} exported to path")

Data from table agg_public_holiday exported to path
Data from table agg_shipments exported to path
Data from table best_performing_product exported to path


In [ ]:


# Specify the local file path to upload
local_file_path = "path/to/local/best_performing_product.csv"

try:
    # Upload the file to S3
    s3.upload_file(local_file_path, bucket_name, key)
    print(f"File uploaded to s3://{bucket_name}/{key}")

except FileNotFoundError:
    print(f"The file {local_file_path} was not found. Please check the file path.")

except (NoCredentialsError, PartialCredentialsError):
    print("Credentials not available. Please provide valid AWS credentials.")

except EndpointConnectionError:
    print("Unable to connect to the S3 endpoint. Please check your network connection and S3 endpoint configuration.")

except (WaiterError, S3UploadFailedError) as e:
    print(f"Error during file upload: {e}")